In [1]:
!pip install torch torchvision numpy matplotlib

In [2]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt


In [3]:

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.GTSRB(root='data', split='train', download=True, transform=transform)
test_data  = datasets.GTSRB(root='data', split='test',  download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_data,  batch_size=64, shuffle=False)


100%|██████████| 187M/187M [00:13<00:00, 13.6MB/s]
100%|██████████| 89.0M/89.0M [00:05<00:00, 17.0MB/s]
100%|██████████| 99.6k/99.6k [00:00<00:00, 194kB/s]


In [4]:

class TrafficSignCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3,32,3,padding=1), nn.ReLU(), nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,3,padding=1), nn.ReLU(), nn.MaxPool2d(2,2),
            nn.Conv2d(64,128,3,padding=1), nn.ReLU(), nn.MaxPool2d(2,2)
        )
        self.fc = nn.Sequential(
            nn.Linear(128*4*4,256), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(256,43)
        )
    def forward(self,x):
        x = self.network(x)
        x = x.view(x.size(0),-1)
        return self.fc(x)


In [5]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TrafficSignCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [6]:

epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss={total_loss/len(train_loader):.4f}")


Epoch 1/10, Loss=2.2063
Epoch 2/10, Loss=0.6454
Epoch 3/10, Loss=0.2418
Epoch 4/10, Loss=0.1308
Epoch 5/10, Loss=0.0902
Epoch 6/10, Loss=0.0643
Epoch 7/10, Loss=0.0498
Epoch 8/10, Loss=0.0416
Epoch 9/10, Loss=0.0371
Epoch 10/10, Loss=0.0342


In [7]:

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        preds = model(images)
        _, predicted = torch.max(preds.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print("Accuracy:", 100 * correct / total)


Accuracy: 91.07680126682502


In [21]:
torch.save(model.state_dict(), "traffic_sign_model.pt")
print("Saved PyTorch model ✔")


Saved PyTorch model ✔


In [22]:
from google.colab import files
files.download("traffic_sign_model.pt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>